# TERA - DSCSP - Aula 28
## Topic Analysis

#### Introdução

Na aula sobre clustering nós utilizamos diversos algoritmos de clustering e redução de dimensionalidade para conseguir encontrar relações de proximidade entre documentos. Entretanto, apesar de algoritmos como o PCA conseguirem representar reduzidamente o nosso conjunto de documentos, nós não conseguíamos interpretar o resultado obtido. Isso acontece porque o PCA encontra novos vetores de features que são combinações lineares do conjunto de palavras existentes. Esse fator pode não ser um problema se o que se deseja é apenas encontrar clusters sem interpretações mais profundas. Entretanto, muitas vezes gostaríamos de entender o racional por trás da geração dos clusters. Ainda mais, as vezes gostaríamos de reduzir um documento a um conjunto de palavras-chave que podem "resumir" o nosso documento e agrupá-las em **tópicos**. E é exatamente esse o objetivo dessa aula.

A área de topic analysis é de grande importância para Machine Learning, ou mais especificamente a área de Data Mining. A utilização de tópicos nos permite ter uma melhor e mais compacta representação dos nossos dados, principalmente quando temos um conjunto extenso de dados e atributos (features). 

Utilizar topic analysis em Natural Language Processing (NLP) é algo bem intuitivo. Nós naturalmente fazemos isso quando queremos organizar textos (documentos) em diferentes categorias, ou temas. Por exemplo, nós podemos ler artigos do Google News e dizer facilmente que um determinado artigo tem o tema "esporte", ou o tema "política". Nosso trabalho em topic analysis é o de conseguir desenvolver algoritmos de Machine Learning que possam encontrar automaticamente esses tópicos, ou temas, por nós.

Vamos definir a seguir alguns termos importantes que serão utilizados daqui por diante:
- **documentos**: são conjuntos de atributos (normalmente palavras) associadas a amostras de uma população (ex: artigos do wikipedia, texto de um livro etc)
- **atributos** (features): é o conjunto de variáveis observadas. Normalmente é um conjunto de palavras que compõe o vocabulário utilizado.
- **variável latente**: variáveis, ou atributos, implícitas no sistema. No nosso caso, podem representar os tópicos dos documentos.
- **vetor de atributos**: é a representação de um determinado documento a partir dos atributos pertencentes a ele
- **matriz de frequência de termos**: é o empilhamento de diversos vetores de atributos associados a cada documento. Cada documento representa uma linha na matriz, enquanto as colunas representam os atributos dos documentos.

<img src="../imagens/mat_freq.png" alt="Drawing" style="width: 500px;"/>

Vamos começar a praticar!

Primeiro vamos realizar os imports necessários

In [1]:
# Imports usados no curso
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks")
plt.rcParams['figure.figsize'] = (12.0, 8.0)

Obs: Lembre-se de colocar os datasets baixados dentro de uma pasta "datasets" na raiz da pasta clonada do repositório da aula!

In [2]:
# Pasta contendo os dados:
DATASET_FOLDER = '../datasets/'

Nós vamos começar utilizando o conhecido dataset de artigos do Wikipedia. Para começar leve, vamos replicar uma parte do código.

In [5]:
# Abra o dataset
df_wiki = pd.read_csv(os.path.join(DATASET_FOLDER, 'wikipedia_dataset_60.csv'), sep=',', names=['titulo', 'artigo', 'cluster'])

df_wiki.head(5)

,titulo,artigo,cluster
0,Black Sabbath,"Black Sabbath are an English rock band, formed...",Music
1,Lymphoma,Lymphoma is a type of blood cancer that occurs...,Sickness
2,Hepatitis C,Hepatitis C is an infectious disease affecting...,Sickness
3,HTTP cookie,"A cookie, also known as an HTTP cookie, web co...",Internet
4,Global warming,Global warming is the rise in the average temp...,Global_Warming


Lembre que os clusters indicados foram feitos apenas para fins didáticos. Na maioria das vezes nós não teremos informações a respeito da relação entre documentos. Realizar esse processo seria trabalhoso demais para a maioria das situações envolvendo Machine Learning. E é exatamente esse tipo de trabalho que queremos automatizar.

In [6]:
# Número aproximado de clusters
n_clusters = len(pd.unique(df_wiki['cluster']))
n_clusters

6

In [13]:
# Exemplo de um artigo:
# Vamos mostrar apenas 1000 caracteres
print("Black Sabbath: \n{} (...)".format(df_wiki[df_wiki.titulo=='Black Sabbath']['artigo'].values[0][:1000]))

Black Sabbath: 
Black Sabbath are an English rock band, formed in Birmingham in 1968, by guitarist Tony Iommi, bassist Geezer Butler, singer Ozzy Osbourne, and drummer Bill Ward. The band has since experienced multiple line-up changes, with Tony Iommi the only constant presence in the band through the years. Originally formed in 1968 as a heavy blues rock band named Earth, the band began incorporating occult themes with horror-inspired lyrics and tuned-down guitars. Despite an association with occult and horror themes, Black Sabbath also composed songs dealing with social instability, political corruption, the dangers of drug abuse and apocalyptic prophecies of the horrors of war. Osbourne's heavy drug use led to his dismissal from the band in 1979. He was replaced by former Rainbow vocalist Ronnie James Dio. After a few albums with Dio's vocals and songwriting collaborations, Black Sabbath endured a revolving line-up in the 1980s and '90s that included vocalists Ian Gillan, Glenn Hugh

Vamos criar agora um embedding para esse texto. Nós utilizaremos uma abordagem de Bag-of-Words (BOW) para esse problema. Mais especificamente, vamos utilizar o Tf-Idf com um tamanho de vocabulário de 15000 palavras.

In [101]:
# Importe o método TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Crie o vetor de embeddings Tf-Idf
# Vamos definir o número máximo de palavras do nosso dicionário (número de dimensões)
# igual a 15000. 
# Também utilizamos um corte em termos muito frequentes em um dado
# documento: max_df=0.8
# Igualmente, realizamos um corte de termos muito pouco frequentes: min_df=0.01
# O parâmetro sublinear_tf utiliza a função 1+log(tf) em vez de uma função linear
# para calcular o peso da frequência de cada termo. Isso permite uma função mais "suave"
# use_idf: Utiliza o inverso da frequência do documento para recriar os pesos da matriz
tfidf = TfidfVectorizer(max_df=0.8, min_df=0.01, max_features=15000, sublinear_tf=True, use_idf=True)

# Precisamos extrair os artigos e títulos do dataframe
titles = df_wiki['titulo'].values
articles = df_wiki['artigo'].values

# Aplique a transformação nos artigos
X = tfidf.fit_transform(articles)

# Tamanho do dataset
X.shape

(58, 15000)

Veja algumas palavras que fazem parte do nosso vocabulário.

In [26]:
list(tfidf.vocabulary_.keys())[:20]

['coldfield',
 'vml',
 'música',
 'directional',
 'feeling',
 'india',
 'tussauds',
 'examination',
 'tolson',
 'insurer',
 'animated',
 'fourier',
 'fertile',
 'presents',
 'baseball',
 'anywhere',
 'agrees',
 'rescuing',
 'seven',
 'freddie']

Note que temos 58 documentos (número de linhas) por 15000 atributos (palavras - colunas). Essa matriz é o que definimos anteriormente por **matriz de frequência de palavras**.

Como temos uma dimensão muito elevada, nós podemos realizar algumas alternativas para reduzir a dimensionalidade. A primeira alternativa é o PCA, mas, como já dissemos anteriormente, ele não nos permite ter uma interpretação dos resultados. Portanto, precisamos de outras alternativas.

## Non-Negative Matrix Factorization (NMF)

O [NMF](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization) é um algoritmo poderoso (apesar de relativamente simples) para encontrar tópicos em um conjunto de documentos e features. Ele se baseia em um processo de decomposição de matrizes para criar uma representação adequada da matriz de frequência de palavras (denotado por **A**). Mais especificamente, o NMF decompõe a matriz de frequência de palavras em duas: a primeira é a matriz de pesos (chamada de **W**), com as linhas representando os documentos e as colunas indicando os tópicos; e a segunda é a matriz de atributos (chamada de **H**), com as linhas indicando os tópicos e as colunas os atributos. O número de tópicos é definido antecipadamente e é fixo.

<img src="../imagens/nmf_draw.png" alt="Drawing" style="width: 500px;"/>

As duas matrizes são formadas a partir de um processo iterativo de otimização (veja esse [link](http://www.columbia.edu/~jwp2128/Teaching/E4903/papers/nmf_nature.pdf) para mais detalhes) com o objetivo de reconstruir fielmente a matriz **A**. Entretanto, para esse fim, a matriz **A** não pode possuir entradas negativas.

Vamos aplicar esse método no problema dos artigos do Wikipedia!

In [30]:
# Primeiro, importe o módulo NMF do scikit-learn
from sklearn.decomposition import NMF

# Precisamos criar a instância do NMF
# Temos que definir um número de componentes para o NMF
# Como temos 6 clusters, vamos escolher n_components=6
nmf = NMF(n_components=6)

# Agora vamos utilizar os mesmos atributos fit, transform ou fit_transform
# que já conhecemos do universo do sklearn
W_nmf = nmf.fit_transform(X)

# Vamos ver qual é a dimensão de W_nmf
W_nmf.shape

(58, 6)

Note que o número de linhas se manteve em 58, que é o número de documentos (artigos) que nós temos, e o número de colunas se transformou em 6, que é o número de tópicos que nós escolhemos. Essa matriz gerada representa a matriz **W** (matriz de pesos) da fatoração de matrizes.

Vamos agora achar a matriz **H** que representa a matriz de atributos.

In [33]:
nmf.components_.shape

(6, 15000)

Note que o número de linhas é igual ao número de tópicos e o número de colunas representa o número de palavras no nosso vocabulário. Cada linha da matriz é definida como um componente (assim como o PCA possui os componentes principais) que está associado a um tópico específico. Entretanto, diferentemente do PCA, nós podemos associar cada componente a um conjunto específico de palavras. Vamos verificar abaixo:

In [57]:
# Precisamos criar uma lista de palavras que representam as 
# colunas da matriz de frequência de palavras
words = [x[0] for x in sorted(tfidf.vocabulary_.items())]

# Vamos criar um dataframe para visualizar
components_df = pd.DataFrame(nmf.components_, columns=words)

# Vamos verificar as palavras que representam cada um dos tópicos
for i in range(6):
    component = components_df.iloc[i]
    print("Topico {}:".format(i))
    print("----------")
    print(component.nlargest())
    print("----------")

Topico 0:
----------
film       0.229489
she        0.216039
starred    0.190290
her        0.186475
award      0.161707
Name: 0, dtype: float64
----------
Topico 1:
----------
treatment    0.162089
disease      0.138412
symptoms     0.133852
infection    0.130183
blood        0.120872
Name: 1, dtype: float64
----------
Topico 2:
----------
cup       0.131772
scored    0.126316
fifa      0.116954
goals     0.113098
team      0.108376
Name: 2, dtype: float64
----------
Topico 3:
----------
climate       0.229481
emissions     0.189357
conference    0.131514
greenhouse    0.125997
change        0.121090
Name: 3, dtype: float64
----------
Topico 4:
----------
users     0.160605
web       0.158407
search    0.152476
google    0.147216
user      0.141580
Name: 4, dtype: float64
----------
Topico 5:
----------
album    0.163353
band     0.144849
song     0.111544
tour     0.104336
songs    0.090645
Name: 5, dtype: float64
----------


O que achou da distribuição de palavras dentro de cada tópico? Acha que faz sentido com os temas principais dos artigos do Wikipedia? Cada um dos tópicos poderia ser associado a um cluster?

Podemos ainda verificar quais são os tópicos principais de alguns artigos específicos.

In [65]:
# Precisamos criar um dataframe para facilitar nossa vida
df = pd.DataFrame(W_nmf, index=titles)

print(df.loc['Denzel Washington'])
print()
print(df.loc['Leukemia'])
print()
print(df.loc['Neymar'])
print()
print(df.loc['LinkedIn'])
print()
print(df.loc['Arctic Monkeys'])

0    0.313460
1    0.000000
2    0.017112
3    0.003640
4    0.000000
5    0.002074
Name: Denzel Washington, dtype: float64

0    0.005121
1    0.495843
2    0.000447
3    0.000000
4    0.000000
5    0.000000
Name: Leukemia, dtype: float64

0    0.016562
1    0.000000
2    0.513333
3    0.000000
4    0.000000
5    0.034787
Name: Neymar, dtype: float64

0    0.024219
1    0.000000
2    0.035836
3    0.043306
4    0.377178
5    0.010995
Name: LinkedIn, dtype: float64

0    0.000000
1    0.000000
2    0.009951
3    0.000000
4    0.019734
5    0.588312
Name: Arctic Monkeys, dtype: float64


Podemos notar que os artigos possuem tópicos coerentes com o que esperávamos!

Vamos agrupar agora os artigos pelos tópicos principais de cada um deles e ver como eles tém relação com os clusters definidos anteriormente.

In [81]:
# Cria as labels a partir do tópico mais relevante de cada artigo
labels = np.argmax(W_nmf, axis=1)

# Cria o novo dataframe com os labels dos clusters
df = pd.DataFrame({'label': labels, 'article': titles})

# Apresenta os resultados
print(df.sort_values(by='label'))

                                          article  label
10                                   Jessica Biel      0
19                                 Angelina Jolie      0
18                                     Mila Kunis      0
38                                  Anne Hathaway      0
5                            Catherine Zeta-Jones      0
46                             Michael Fassbender      0
7                               Denzel Washington      0
50                               Jennifer Aniston      0
25                                 Dakota Fanning      0
12                                  Russell Crowe      0
49                                    Tonsillitis      1
31                                       Leukemia      1
24                                          Fever      1
52                                     Prednisone      1
9                                            Gout      1
8                                     Hepatitis B      1
34                             

O que achou do resultado? Percebeu que o NMF não só encontrou uma representação em tópicos dos documentos, mas também teve um papel de agregador? Ele realizou um ótimo trabalho em encontrar clusters. E o melhor, nós podemos explicar com palavras o que representa cada um dos tópicos/clusters.

### Exercício

Vamos praticar um pouco com o NMF. O dataset que vamos utilizar é um dataset padrão do scikit-learn que é muito útil para algoritmos de NLP. O dataset contém grupos de discussão no [Usenet](https://en.wikipedia.org/wiki/Usenet) com 18.000 postagens e 20 tópicos principais.

In [91]:
# Importe o dataset
from sklearn.datasets import fetch_20newsgroups

# Vamos escolher apenas algumas categorias para facilitar
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# Pegamos apenas o corpo do texto
dataset = fetch_20newsgroups(shuffle=True, random_state=1, categories=categories,
                             remove=('headers', 'footers', 'quotes'))

# Para limitar um pouco a quantidade de dados, vamos limitar o dataset
X = dataset.data[:2000]

In [102]:
# Exemplos de documentos
print("\n".join(data_samples[:1]))

Rick Anderson replied to my letter with...

ra> In article <C5ELp2.L0C@acsu.buffalo.edu>,
ra>
ra> >     Well, Jason, it's heretical in a few ways. The first point is that
ra> >     this equates Lucifer and Jesus as being the same type of being.
ra> >     However, Lucifer is a created being: "Thou [wast] perfect in thy
ra> >     ways from the day that thou wast created, till iniquity was found in
ra> >     thee." (Ezekiel 28:15). While Jesus is uncreated, and the Creator of
ra> >     all things: "In the beginning was the Word, and the Word was with
ra> >     God, and the Word was God.  The same was in the beginning with God.
ra> >     All things were made by him; and without him was not any thing made
ra> >     that was made." (John 1:1-3) "And he is before all things, and by
ra> >     him all things consist." (Colossians 1:17)
ra>
ra>    Your inference from the Ezekiel and John passages that Lucifer was
ra> "created" and that Jesus was not depends on a particular interpetation of
ra> t

Percebemos que os dados estão um pouco sujos e tem diversas palavras que podem não significar muito para nós.

Crie o vetor de Bag-of-Words Tf-Idf a partir dos documentos.

In [ ]:
# TODO:
# Crie a matriz de frequência de palavras utilizando Tf-Idf
# Dica: 